# 라이브러리 로드

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

import warnings
from pathlib import Path

from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

import xgboost as xgb

from hyperopt import hp 
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials, space_eval

In [3]:
pd.set_option('max_columns', 100)
pd.set_option('display.precision', 4)

warnings.filterwarnings('ignore')

# 학습데이터 로드

In [4]:
data_dir = Path('../data/dacon-novel-author-classification')
feature_dir = Path('../build/feature')
val_dir = Path('../build/val')
tst_dir = Path('../build/tst')
sub_dir = Path('../build/sub')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test_x.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5
seed = 2020

In [5]:
algorithm_name = 'xgb'

feature_names= ['stacking-layer1-all']

feature_target_file = feature_dir / f'feature_target.csv'

model_names = []
for feature_name in feature_names:
    model_names.append(f'{algorithm_name}_{feature_name}')
    
stacking_oof_pred_files=[]
for model_name in model_names:
    stacking_oof_pred_files.append( val_dir / f'{model_name}_oof_pred.csv')
    
stacking_test_pred_files=[]
for model_name in model_names:
    stacking_test_pred_files.append( tst_dir / f'{model_name}_test_pred.csv')
    
stacking_submission_files=[]
for model_name in model_names:
    stacking_submission_files.append( sub_dir / f'{model_name}_submission.csv')

# Stacking feature 생성

In [6]:
def load_feature(model_names, number_of_ver=None, kind=None):
    oof_list = []
    test_list = []
    
    if number_of_ver==None or kind==None:
        print('error')
        return None
    
    # 딥러닝 시리즈 4가지 버전
    if kind == 0:
        for model in model_names:
            print(f'load {model}_cv')
            for i in range(1,number_of_ver+1):
                oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv', delimiter=','))
                test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
    
    # 로지스틱 회귀 6가지 버전
    elif kind == 1:
        for model in model_names:
            print(f'load {model}_cv')
            for i in range(1, number_of_ver+1):
                oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv', delimiter=','))
                test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))

    # 신경망 기반 불용어 처리 21가지 버전 또는 머신러닝 기반 불용어 처리 18가지 버전
    elif kind == 2:
        for model in model_names:
            print(f'load {model}_cv')
            if model.find('feature') != -1:
                for i in range(2,5):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('tfidf') != -1:
                for i in range(1,4):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('hashing') != -1:
                for i in range(1,4):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('bow') != -1:
                for i in range(1,4):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            else:
                print('not found')
    
    # 신경만 기반 불용어 처리 X 13가지 버전 또는 머신러닝 기반 불용어 처리 X 18가지 버전
    elif kind == 3:
        for model in model_names:
            print(f'load {model}_cv')
            if model.find('feature') != -1:
                for i in range(1,2):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('tfidf') != -1:
                for i in range(4,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('hashing') != -1:
                for i in range(4,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('bow') != -1:
                for i in range(4,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            else:
                print('not found')
                
    # 첫번째 레이어를 학습하기 위한 데이터셋 모두 가져오기 버전
    elif kind == 4:
        for model in model_names:
            print(f'load {model}_cv')
            if model.find('feature') != -1:
                for i in range(1,5):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('tfidf') != -1:
                for i in range(1,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('hashing') != -1:
                for i in range(1,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('bow') != -1:
                for i in range(1,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            else:
                print('not found')
    
    # 두번째 레이어를 학습하기 위한 데이터셋 모두 가져오기 버전
    elif kind == 5:
        for model in model_names:
            print(f'load {model}_cv')
            if model.find('stacking') != -1:
                for feature in ['stopwords-yes-nn','stopwords-no-nn','stopwords-no-ml', 'stopwords-no-ml'] :
                    oof_list.append(np.loadtxt(val_dir / f'{model}-{feature}_oof_pred.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}-{feature}_test_pred.csv',delimiter=','))
            elif model.find('all') != -1:
                oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred.csv',delimiter=','))
                test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred.csv',delimiter=','))
    
    # 두번째 레이어를 학습하기 위한 PCA 데이터셋 + xgb/lgbm tfidf 데이터셋 가져오기 버전 - 하나씩
    elif kind ==6:
        for model in model_names:
            print(f'load {model}_cv')
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred.csv', delimiter=','))
            test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred.csv', delimiter=','))
            
                        
        
    
    return oof_list, test_list

In [7]:
model_names = ['mlp_tfidf-pca', 'lr_tfidf-pca', 'cnn_tfidf-pca', 'lgbm_tfidf','lgbm_tfidf-pca']

tmp_oof, tmp_test = load_feature(model_names, -1, 6)
tmp_oof = np.concatenate(tmp_oof, axis=1)
tmp_test = np.concatenate(tmp_test, axis=1)
print(f'shape : {tmp_oof.shape}, {tmp_test.shape}')

load mlp_tfidf-pca_cv
load lr_tfidf-pca_cv
load cnn_tfidf-pca_cv
load lgbm_tfidf_cv
load lgbm_tfidf-pca_cv
shape : (54879, 25), (19617, 25)


In [8]:
model_names = ['cnn_bow','cnn_feature','cnn_hashing','cnn_tfidf','lr_bow','lr_hashing','lr_tfidf','lstm_feature',
              'mlp_bow','mlp_feature','mlp_hashing','mlp_tfidf','transformer_feature','transformer_v2_feature']

all_oof, all_test = load_feature(model_names, -1, 4)
all_oof = np.concatenate(all_oof, axis=1)
all_test = np.concatenate(all_test, axis=1)
print(f'nn_yes shape : {all_oof.shape}, {all_test.shape}')

load cnn_bow_cv
load cnn_feature_cv
load cnn_hashing_cv
load cnn_tfidf_cv
load lr_bow_cv
load lr_hashing_cv
load lr_tfidf_cv
load lstm_feature_cv
load mlp_bow_cv
load mlp_feature_cv
load mlp_hashing_cv
load mlp_tfidf_cv
load transformer_feature_cv
load transformer_v2_feature_cv
nn_yes shape : (54879, 370), (19617, 370)


In [9]:
all_oof = np.concatenate([all_oof, tmp_oof], axis=1)
all_test = np.concatenate([all_test, tmp_test], axis=1)
print(f'shape : {all_oof.shape}, {all_test.shape}')

shape : (54879, 395), (19617, 395)


In [10]:
y = pd.read_csv(feature_target_file, index_col=0, usecols=['index',target_col]).values.flatten()
y.shape

(54879,)

# 하이퍼파라미터 튜닝

In [11]:
# XGB parameters

xgb_reg_params = {
    'n_jobs': -1,
    'n_estimators': 1000,
    'eval_metric': 'mlogloss',
    'objective': 'multi:softprob',
    'num_class': 5,
    'random_state': 2020,
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',

    'booster': 'gbtree',

    'max_depth': hp.choice('max_depth',np.arange(5,30,1,dtype=int)),
    'alpha': hp.uniform('alpha', 0.01, 0.4),
    'lambda': hp.uniform('lambda', 0.01, 0.4),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 0.9),
    'gamma': hp.uniform('gamma', 0.01, 0.7),
    'max_leaves': hp.choice('max_leaves', np.arange(20,250,1,dtype=int)),
    'subsample': hp.choice('subsample', [0.2,0.4,0.5,0.6,0.7,0.8,0.9]),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.4, 0.8),
}

xgb_fit_params = {
    'eval_metric': 'mlogloss',
    'early_stopping_rounds': 50,
    'verbose': True
}

xgb_para = dict()
xgb_para['reg_params'] = xgb_reg_params
xgb_para['fit_params'] = xgb_fit_params
xgb_para['loss_func'] = lambda y, pred: log_loss(y,pred, labels=[0,1,2,3,4]) 

In [12]:
class HPOpt(object):

    def __init__(self, X_train, X_val, y_train, y_val):
        self.X_train = X_train
        self.X_val = X_val
        self.y_train = y_train
        self.y_val = y_val

    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result

    def xgb_clf(self, para):
        clf = xgb.XGBClassifier(**para['reg_params'])
        return self.train_reg(clf, para)
    
    def train_reg(self, clf, para):
        clf.fit(self.X_train, self.y_train,
                eval_set=[(self.X_train, self.y_train), (self.X_val, self.y_val)],
                **para['fit_params'])
        
        loss = clf.best_score
        return {'loss': loss, 'status': STATUS_OK, 'model': clf}

In [13]:
X_train, X_val, y_train, y_val = train_test_split(all_oof, y, test_size= 0.25, random_state=2020)

obj = HPOpt(X_train, X_val, y_train, y_val)

trials=Trials()
best = obj.process(fn_name='xgb_clf', space=xgb_para, trials=trials, algo=tpe.suggest, max_evals=30)

[0]	validation_0-mlogloss:1.44766	validation_1-mlogloss:1.44863

Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.


Will train until validation_1-mlogloss hasn't improved in 50 rounds.

[1]	validation_0-mlogloss:1.31786	validation_1-mlogloss:1.32052

[2]	validation_0-mlogloss:1.21106	validation_1-mlogloss:1.21564

[3]	validation_0-mlogloss:1.11982	validation_1-mlogloss:1.12570

[4]	validation_0-mlogloss:1.04111	validation_1-mlogloss:1.04857

[5]	validation_0-mlogloss:0.97267	validation_1-mlogloss:0.98162

[6]	validation_0-mlogloss:0.91229	validation_1-mlogloss:0.92266

[7]	validation_0-mlogloss:0.85896	validation_1-mlogloss:0.87086

[8]	validation_0-mlogloss:0.81165	validation_1-mlogloss:0.82479

[9]	validation_0-mlogloss:0.76932	validation_1-mlogloss:0.78374

[10]	validation_0-mlogloss:0.73145	validation_1-mlogloss:0.74710

[11]	validation_0-mlogloss:0.69725	validation_1-mlogloss:0.71419

[12]	validation_0-mlogloss:0.66640	validation_1-m

[121]	validation_0-mlogloss:0.21945	validation_1-mlogloss:0.38692

[122]	validation_0-mlogloss:0.21836	validation_1-mlogloss:0.38696

[123]	validation_0-mlogloss:0.21723	validation_1-mlogloss:0.38702

[124]	validation_0-mlogloss:0.21602	validation_1-mlogloss:0.38689

[125]	validation_0-mlogloss:0.21492	validation_1-mlogloss:0.38679

[126]	validation_0-mlogloss:0.21379	validation_1-mlogloss:0.38674

[127]	validation_0-mlogloss:0.21271	validation_1-mlogloss:0.38676

[128]	validation_0-mlogloss:0.21151	validation_1-mlogloss:0.38678

[129]	validation_0-mlogloss:0.21040	validation_1-mlogloss:0.38684

[130]	validation_0-mlogloss:0.20928	validation_1-mlogloss:0.38673

[131]	validation_0-mlogloss:0.20816	validation_1-mlogloss:0.38668

[132]	validation_0-mlogloss:0.20702	validation_1-mlogloss:0.38670

[133]	validation_0-mlogloss:0.20586	validation_1-mlogloss:0.38674

[134]	validation_0-mlogloss:0.20472	validation_1-mlogloss:0.38666

[135]	validation_0-mlogloss:0.20348	validation_1-mlogloss:0.38

[49]	validation_0-mlogloss:0.26139	validation_1-mlogloss:0.39143     

[50]	validation_0-mlogloss:0.25900	validation_1-mlogloss:0.39134     

[51]	validation_0-mlogloss:0.25669	validation_1-mlogloss:0.39098     

[52]	validation_0-mlogloss:0.25429	validation_1-mlogloss:0.39097     

[53]	validation_0-mlogloss:0.25200	validation_1-mlogloss:0.39086     

[54]	validation_0-mlogloss:0.24947	validation_1-mlogloss:0.39046     

[55]	validation_0-mlogloss:0.24698	validation_1-mlogloss:0.39021     

[56]	validation_0-mlogloss:0.24456	validation_1-mlogloss:0.39025     

[57]	validation_0-mlogloss:0.24225	validation_1-mlogloss:0.39023     

[58]	validation_0-mlogloss:0.24010	validation_1-mlogloss:0.39024     

[59]	validation_0-mlogloss:0.23789	validation_1-mlogloss:0.39016     

[60]	validation_0-mlogloss:0.23555	validation_1-mlogloss:0.39029     

[61]	validation_0-mlogloss:0.23358	validation_1-mlogloss:0.39013     

[62]	validation_0-mlogloss:0.23120	validation_1-mlogloss:0.39001     

[63]	v

[29]	validation_0-mlogloss:0.27926	validation_1-mlogloss:0.40051     

[30]	validation_0-mlogloss:0.27418	validation_1-mlogloss:0.39909     

[31]	validation_0-mlogloss:0.26974	validation_1-mlogloss:0.39812     

[32]	validation_0-mlogloss:0.26502	validation_1-mlogloss:0.39709     

[33]	validation_0-mlogloss:0.26072	validation_1-mlogloss:0.39615     

[34]	validation_0-mlogloss:0.25612	validation_1-mlogloss:0.39563     

[35]	validation_0-mlogloss:0.25186	validation_1-mlogloss:0.39496     

[36]	validation_0-mlogloss:0.24774	validation_1-mlogloss:0.39450     

[37]	validation_0-mlogloss:0.24337	validation_1-mlogloss:0.39399     

[38]	validation_0-mlogloss:0.23919	validation_1-mlogloss:0.39371     

[39]	validation_0-mlogloss:0.23502	validation_1-mlogloss:0.39351     

[40]	validation_0-mlogloss:0.23109	validation_1-mlogloss:0.39304     

[41]	validation_0-mlogloss:0.22767	validation_1-mlogloss:0.39254     

[42]	validation_0-mlogloss:0.22386	validation_1-mlogloss:0.39237     

[43]	v

[44]	validation_0-mlogloss:0.42342	validation_1-mlogloss:0.50604     

[45]	validation_0-mlogloss:0.41660	validation_1-mlogloss:0.50073     

[46]	validation_0-mlogloss:0.41009	validation_1-mlogloss:0.49569     

[47]	validation_0-mlogloss:0.40382	validation_1-mlogloss:0.49092     

[48]	validation_0-mlogloss:0.39771	validation_1-mlogloss:0.48635     

[49]	validation_0-mlogloss:0.39193	validation_1-mlogloss:0.48200     

[50]	validation_0-mlogloss:0.38638	validation_1-mlogloss:0.47799     

[51]	validation_0-mlogloss:0.38105	validation_1-mlogloss:0.47411     

[52]	validation_0-mlogloss:0.37588	validation_1-mlogloss:0.47046     

[53]	validation_0-mlogloss:0.37084	validation_1-mlogloss:0.46683     

[54]	validation_0-mlogloss:0.36590	validation_1-mlogloss:0.46336     

[55]	validation_0-mlogloss:0.36117	validation_1-mlogloss:0.46010     

[56]	validation_0-mlogloss:0.35658	validation_1-mlogloss:0.45697     

[57]	validation_0-mlogloss:0.35211	validation_1-mlogloss:0.45396     

[58]	v

...

[278]	validation_0-mlogloss:0.17468	validation_1-mlogloss:0.38651      

[279]	validation_0-mlogloss:0.17410	validation_1-mlogloss:0.38648      

[280]	validation_0-mlogloss:0.17352	validation_1-mlogloss:0.38647      

[281]	validation_0-mlogloss:0.17296	validation_1-mlogloss:0.38648      

[282]	validation_0-mlogloss:0.17236	validation_1-mlogloss:0.38650      

[283]	validation_0-mlogloss:0.17175	validation_1-mlogloss:0.38650      

[284]	validation_0-mlogloss:0.17117	validation_1-mlogloss:0.38645      

[285]	validation_0-mlogloss:0.17067	validation_1-mlogloss:0.38642      

[286]	validation_0-mlogloss:0.17011	validation_1-mlogloss:0.38643      

[287]	validation_0-mlogloss:0.16952	validation_1-mlogloss:0.38643      

[288]	validation_0-mlogloss:0.16892	validation_1-mlogloss:0.38648      

[289]	validation_0-mlogloss:0.16835	validation_1-mlogloss:0.38653      

[290]	validation_0-mlogloss:0.16778	validation_1-mlogloss:0.38650      

[291]	validation_0-mlogloss:0.16724	validation_1-ml

[18]	validation_0-mlogloss:0.41403	validation_1-mlogloss:0.52032       

[19]	validation_0-mlogloss:0.39710	validation_1-mlogloss:0.50775       

[20]	validation_0-mlogloss:0.38160	validation_1-mlogloss:0.49663       

[21]	validation_0-mlogloss:0.36693	validation_1-mlogloss:0.48630       

[22]	validation_0-mlogloss:0.35339	validation_1-mlogloss:0.47735       

[23]	validation_0-mlogloss:0.34073	validation_1-mlogloss:0.46914       

[24]	validation_0-mlogloss:0.32908	validation_1-mlogloss:0.46148       

[25]	validation_0-mlogloss:0.31820	validation_1-mlogloss:0.45460       

[26]	validation_0-mlogloss:0.30790	validation_1-mlogloss:0.44850       

[27]	validation_0-mlogloss:0.29823	validation_1-mlogloss:0.44324       

[28]	validation_0-mlogloss:0.28900	validation_1-mlogloss:0.43810       

[29]	validation_0-mlogloss:0.28056	validation_1-mlogloss:0.43353       

[30]	validation_0-mlogloss:0.27273	validation_1-mlogloss:0.42962       

[31]	validation_0-mlogloss:0.26510	validation_1-mlo

[10]	validation_0-mlogloss:0.96463	validation_1-mlogloss:0.98035       

[11]	validation_0-mlogloss:0.93000	validation_1-mlogloss:0.94692       

[12]	validation_0-mlogloss:0.89736	validation_1-mlogloss:0.91555       

[13]	validation_0-mlogloss:0.86698	validation_1-mlogloss:0.88637       

[14]	validation_0-mlogloss:0.83852	validation_1-mlogloss:0.85916       

[15]	validation_0-mlogloss:0.81160	validation_1-mlogloss:0.83342       

[16]	validation_0-mlogloss:0.78636	validation_1-mlogloss:0.80941       

[17]	validation_0-mlogloss:0.76275	validation_1-mlogloss:0.78710       

[18]	validation_0-mlogloss:0.74038	validation_1-mlogloss:0.76584       

[19]	validation_0-mlogloss:0.71940	validation_1-mlogloss:0.74599       

[20]	validation_0-mlogloss:0.69959	validation_1-mlogloss:0.72744       

[21]	validation_0-mlogloss:0.68072	validation_1-mlogloss:0.70967       

[22]	validation_0-mlogloss:0.66308	validation_1-mlogloss:0.69312       

[23]	validation_0-mlogloss:0.64629	validation_1-mlo

[122]	validation_0-mlogloss:0.25129	validation_1-mlogloss:0.38931      

[123]	validation_0-mlogloss:0.25012	validation_1-mlogloss:0.38919      

[124]	validation_0-mlogloss:0.24907	validation_1-mlogloss:0.38902      

[125]	validation_0-mlogloss:0.24787	validation_1-mlogloss:0.38893      

[126]	validation_0-mlogloss:0.24675	validation_1-mlogloss:0.38875      

[127]	validation_0-mlogloss:0.24569	validation_1-mlogloss:0.38865      

[128]	validation_0-mlogloss:0.24452	validation_1-mlogloss:0.38854      

[129]	validation_0-mlogloss:0.24334	validation_1-mlogloss:0.38841      

[130]	validation_0-mlogloss:0.24224	validation_1-mlogloss:0.38829      

[131]	validation_0-mlogloss:0.24104	validation_1-mlogloss:0.38821      

[132]	validation_0-mlogloss:0.23985	validation_1-mlogloss:0.38813      

[133]	validation_0-mlogloss:0.23866	validation_1-mlogloss:0.38801      

[134]	validation_0-mlogloss:0.23755	validation_1-mlogloss:0.38779      

[135]	validation_0-mlogloss:0.23638	validation_1-ml

[234]	validation_0-mlogloss:0.14931	validation_1-mlogloss:0.38643      

[235]	validation_0-mlogloss:0.14860	validation_1-mlogloss:0.38642      

[236]	validation_0-mlogloss:0.14791	validation_1-mlogloss:0.38648      

[237]	validation_0-mlogloss:0.14724	validation_1-mlogloss:0.38649      

[238]	validation_0-mlogloss:0.14656	validation_1-mlogloss:0.38654      

[239]	validation_0-mlogloss:0.14585	validation_1-mlogloss:0.38657      

[240]	validation_0-mlogloss:0.14521	validation_1-mlogloss:0.38658      

Stopping. Best iteration:                                              
[190]	validation_0-mlogloss:0.18343	validation_1-mlogloss:0.38536


100%|██████████| 30/30 [20:25<00:00, 40.85s/trial, best loss: 0.381039]


In [14]:
best

{'alpha': 0.3976552619317213,
 'colsample_bytree': 0.5503037696353703,
 'gamma': 0.4515313761865322,
 'lambda': 0.3254110339102569,
 'learning_rate': 0.016560280940015843,
 'max_depth': 0,
 'max_leaves': 120,
 'subsample': 5}

In [15]:
hyperparams = space_eval(xgb_reg_params, best)
hyperparams

{'alpha': 0.3976552619317213,
 'booster': 'gbtree',
 'colsample_bytree': 0.5503037696353703,
 'eval_metric': 'mlogloss',
 'gamma': 0.4515313761865322,
 'lambda': 0.3254110339102569,
 'learning_rate': 0.016560280940015843,
 'max_depth': 5,
 'max_leaves': 140,
 'n_estimators': 1000,
 'n_jobs': -1,
 'num_class': 5,
 'objective': 'multi:softprob',
 'predictor': 'gpu_predictor',
 'random_state': 2020,
 'subsample': 0.8,
 'tree_method': 'gpu_hist'}

In [16]:
trials.best_trial['result']['model']

XGBClassifier(alpha=0.3976552619317213, base_score=0.5, booster='gbtree',
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.5503037696353703, eval_metric='mlogloss',
              gamma=0.4515313761865322, gpu_id=0, importance_type='gain',
              interaction_constraints='', lambda=0.3254110339102569,
              learning_rate=0.016560280940015843, max_delta_step=0, max_depth=5,
              max_lea...
              monotone_constraints='(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,